K-fold cross-validationやLeave-one-out cross-validationを行ってみる。

In [1]:
import numpy as np
import pandas as pd
import sys

from sklearn import svm
from sklearn.datasets import load_iris, load_digits
from sklearn.model_selection import KFold, ShuffleSplit, GroupKFold, \
                                                                StratifiedKFold, StratifiedShuffleSplit,  \
                                                                LeaveOneOut, LeavePOut, \
                                                                cross_val_predict, cross_val_score, GridSearchCV


print(sys.version_info)

sys.version_info(major=3, minor=6, micro=2, releaselevel='final', serial=0)


### 1. K-fold cross validationのための様々なクラスや関数

#### 1-1.kFold

[kFold](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold)を使うことでfoldを作成し、Training setとValidation setに分割することができる。<br>
再現性を担保せずに分割したい場合は`random_state=None`, `shuffle=True`とする。  
このルールは他のクラスでも適用される。

In [2]:
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11, 12]])
y = np.array([1, 2, 1, 2, 1, 2])
k_fold = KFold(n_splits=3)
print(k_fold)
for train_index, validation_index in k_fold.split(X):
        print("TRAIN:", train_index, "VALIDATION:", validation_index)
k_fold = KFold(n_splits=3, random_state=None, shuffle=True)
print(k_fold)
for train_index, validation_index in k_fold.split(X):
        print("TRAIN:", train_index, "VALIDATION:", validation_index)    


KFold(n_splits=3, random_state=None, shuffle=False)
TRAIN: [2 3 4 5] VALIDATION: [0 1]
TRAIN: [0 1 4 5] VALIDATION: [2 3]
TRAIN: [0 1 2 3] VALIDATION: [4 5]
KFold(n_splits=3, random_state=None, shuffle=True)
TRAIN: [0 1 3 5] VALIDATION: [2 4]
TRAIN: [1 2 3 4] VALIDATION: [0 5]
TRAIN: [0 2 4 5] VALIDATION: [1 3]


`random_state=123`などのように、intを指定するとseed値を固定することも可能である。  
以下の例では2回3-foldを行っているが`random_state=123`としているので2回のTraining set, Validation setは一致している。

In [3]:
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11, 12]])
y = np.array([1, 2, 1, 2, 1, 2])
k_fold = KFold(n_splits=3)
k_fold = KFold(n_splits=3, random_state=123, shuffle=True)
print(k_fold)
print('1st')
for train_index, validation_index in k_fold.split(X):
        print("TRAIN:", train_index, "VALIDATION:", validation_index)    
print('2nd')
for train_index, validation_index in k_fold.split(X):
        print("TRAIN:", train_index, "VALIDATION:", validation_index)    

KFold(n_splits=3, random_state=123, shuffle=True)
1st
TRAIN: [0 2 4 5] VALIDATION: [1 3]
TRAIN: [1 2 3 5] VALIDATION: [0 4]
TRAIN: [0 1 3 4] VALIDATION: [2 5]
2nd
TRAIN: [0 2 4 5] VALIDATION: [1 3]
TRAIN: [1 2 3 5] VALIDATION: [0 4]
TRAIN: [0 1 3 4] VALIDATION: [2 5]


#### 1-2. ShuffleSplit

[ShuffleSplit](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html#sklearn.model_selection.ShuffleSplit)を使ってもランダムにfoldを作成し、Training setとValidation setに分割することができる。<br>
再現性を担保せずに分割したい場合は`random_state=None`とする。

In [4]:
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11, 12]])
y = np.array([1, 2, 1, 2, 1, 2])
rs = ShuffleSplit(n_splits=3, random_state=None, test_size=0.5, train_size=0.5)
print(rs)
for train_index, validation_index in rs.split(X):
        print("TRAIN:", train_index, "VALIDATION:", validation_index)


ShuffleSplit(n_splits=3, random_state=None, test_size=0.5, train_size=0.5)
TRAIN: [3 4 2] VALIDATION: [0 5 1]
TRAIN: [0 2 5] VALIDATION: [3 1 4]
TRAIN: [5 3 1] VALIDATION: [4 2 0]


#### 1-3. GroupKFold

[GroupKFold](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupKFold.html#sklearn.model_selection.GroupKFold)を使うと必ず同じfoldに入るためのグループを作ることができる。<br>
例えば以下の例では0, 1番目のデータ、2, 3番目のデータはそれぞれ必ず同じfoldに分類される。

In [5]:
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11, 12]])
y = np.array([1, 2, 1, 2, 1, 2])
groups = np.array([0, 0, 0, 2, 2, 2])
skf = group_kfold = GroupKFold(n_splits=2)
group_kfold.get_n_splits(X, y, groups)
print(skf)
for train_index, validation_index in group_kfold.split(X, y, groups):
        print("TRAIN:", train_index, "VALIDATION:", validation_index)
        X_train, X_test = X[train_index], X[validation_index]
        y_train, y_test = y[train_index], y[validation_index]
        print(X_train, X_test, y_train, y_test)


GroupKFold(n_splits=2)
TRAIN: [0 1 2] VALIDATION: [3 4 5]
[[1 2]
 [3 4]
 [5 6]] [[ 7  8]
 [ 9 10]
 [11 12]] [1 2 1] [2 1 2]
TRAIN: [3 4 5] VALIDATION: [0 1 2]
[[ 7  8]
 [ 9 10]
 [11 12]] [[1 2]
 [3 4]
 [5 6]] [2 1 2] [1 2 1]


#### 1-4. StratifiedKFold

[StratifiedKFold](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold)を使うと目的変数のクラスごとの割合を考慮してfoldを作成することができる。<br>以下の例では<br>y=1であるインデックス番号0, 1, 2, 3、<br>y=2であるインデックス番号4, 5, 6, 7<br>から半分ずつ取り出しTraining setとし、残りをValidation setとしfoldを作成している。<br>
再現性を担保せずに分割したい場合は`shuffle=True`とし、seed値を固定しない場合は`random_state=None`とする。

In [6]:
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13, 14], [15, 16]])
y = np.array([1, 1, 1, 1, 2, 2, 2, 2])
skf = StratifiedKFold(n_splits=2)
print(skf)
for train_index, validation_index in skf.split(X, y):
        print("TRAIN:", train_index, "VALIDATION:", validation_index)
        X_train, X_test = X[train_index], X[validation_index]
        y_train, y_test = y[train_index], y[validation_index]
skf = StratifiedKFold(n_splits=4, random_state=None, shuffle=True)
print(skf)
for train_index, validation_index in skf.split(X, y):
        print("TRAIN:", train_index, "VALIDATION:", validation_index)
        X_train, X_test = X[train_index], X[validation_index]
        y_train, y_test = y[train_index], y[validation_index]    

StratifiedKFold(n_splits=2, random_state=None, shuffle=False)
TRAIN: [2 3 6 7] VALIDATION: [0 1 4 5]
TRAIN: [0 1 4 5] VALIDATION: [2 3 6 7]
StratifiedKFold(n_splits=4, random_state=None, shuffle=True)
TRAIN: [0 2 3 4 6 7] VALIDATION: [1 5]
TRAIN: [0 1 3 4 5 7] VALIDATION: [2 6]
TRAIN: [1 2 3 5 6 7] VALIDATION: [0 4]
TRAIN: [0 1 2 4 5 6] VALIDATION: [3 7]


#### 1-5. StratifiedShuffleSplit

[StratifiedShuffleSplit](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html#sklearn.model_selection.StratifiedShuffleSplit)を使ってもランダムにクラスごとの割合を考慮しつつfoldを作成し、<br>Training setとValidation setに分割することができる。<br>
乱数を固定したくない場合はrandom_state=Noneとする。

In [7]:
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13, 14], [15, 16]])
y = np.array([1, 1, 1, 1, 2, 2, 2, 2])
sss = StratifiedShuffleSplit(n_splits=2, test_size=0.5, random_state=0)
print(sss)
for train_index, validation_index in sss.split(X, y):
        print("TRAIN:", train_index, "VALIDATION:", validation_index)
sss = StratifiedShuffleSplit(n_splits=2, test_size=0.25, random_state=None)
print(sss)
for train_index, validation_index in sss.split(X, y):
        print("TRAIN:", train_index, "VALIDATION:", validation_index)

StratifiedShuffleSplit(n_splits=2, random_state=0, test_size=0.5,
            train_size=None)
TRAIN: [6 2 4 3] VALIDATION: [0 1 5 7]
TRAIN: [2 1 4 7] VALIDATION: [3 6 5 0]
StratifiedShuffleSplit(n_splits=2, random_state=None, test_size=0.25,
            train_size=None)
TRAIN: [0 4 1 3 5 7] VALIDATION: [2 6]
TRAIN: [7 5 2 1 4 0] VALIDATION: [3 6]


### 2. Leave-one-out cross validationのための様々なクラスや関数

#### 2-1. LeaveOneOut

[LeaveOneOut](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html#sklearn.model_selection.LeaveOneOut)を使うことでデータセットのサンプル一つずつをValidation setとすることができる。

In [8]:
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13, 14], [15, 16]])
y = np.array([1, 1, 1, 1, 2, 2, 2, 2])
loo = LeaveOneOut()
print(loo)
for train_index, validation_index in loo.split(X):
        print("TRAIN:", train_index, "VALIDATION:", validation_index)

LeaveOneOut()
TRAIN: [1 2 3 4 5 6 7] VALIDATION: [0]
TRAIN: [0 2 3 4 5 6 7] VALIDATION: [1]
TRAIN: [0 1 3 4 5 6 7] VALIDATION: [2]
TRAIN: [0 1 2 4 5 6 7] VALIDATION: [3]
TRAIN: [0 1 2 3 5 6 7] VALIDATION: [4]
TRAIN: [0 1 2 3 4 6 7] VALIDATION: [5]
TRAIN: [0 1 2 3 4 5 7] VALIDATION: [6]
TRAIN: [0 1 2 3 4 5 6] VALIDATION: [7]


#### 2-2. LeavePOut

1個ではなく2個や3個取り出したいときはLeavePOutを使う(P=2, 3など)。<br>
データセットのサンプルP個ずつをValidation setとすることができる。

In [9]:
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13, 14], [15, 16]])
y = np.array([1, 1, 1, 1, 2, 2, 2, 2])
lpo = LeavePOut(2)
print(lpo)
for train_index, validation_index in lpo.split(X):
        print("TRAIN:", train_index, "VALIDATION:", validation_index)

LeavePOut(p=2)
TRAIN: [2 3 4 5 6 7] VALIDATION: [0 1]
TRAIN: [1 3 4 5 6 7] VALIDATION: [0 2]
TRAIN: [1 2 4 5 6 7] VALIDATION: [0 3]
TRAIN: [1 2 3 5 6 7] VALIDATION: [0 4]
TRAIN: [1 2 3 4 6 7] VALIDATION: [0 5]
TRAIN: [1 2 3 4 5 7] VALIDATION: [0 6]
TRAIN: [1 2 3 4 5 6] VALIDATION: [0 7]
TRAIN: [0 3 4 5 6 7] VALIDATION: [1 2]
TRAIN: [0 2 4 5 6 7] VALIDATION: [1 3]
TRAIN: [0 2 3 5 6 7] VALIDATION: [1 4]
TRAIN: [0 2 3 4 6 7] VALIDATION: [1 5]
TRAIN: [0 2 3 4 5 7] VALIDATION: [1 6]
TRAIN: [0 2 3 4 5 6] VALIDATION: [1 7]
TRAIN: [0 1 4 5 6 7] VALIDATION: [2 3]
TRAIN: [0 1 3 5 6 7] VALIDATION: [2 4]
TRAIN: [0 1 3 4 6 7] VALIDATION: [2 5]
TRAIN: [0 1 3 4 5 7] VALIDATION: [2 6]
TRAIN: [0 1 3 4 5 6] VALIDATION: [2 7]
TRAIN: [0 1 2 5 6 7] VALIDATION: [3 4]
TRAIN: [0 1 2 4 6 7] VALIDATION: [3 5]
TRAIN: [0 1 2 4 5 7] VALIDATION: [3 6]
TRAIN: [0 1 2 4 5 6] VALIDATION: [3 7]
TRAIN: [0 1 2 3 6 7] VALIDATION: [4 5]
TRAIN: [0 1 2 3 5 7] VALIDATION: [4 6]
TRAIN: [0 1 2 3 5 6] VALIDATION: [4 7]
TRAIN: [0 

### 3. k-Fold や Leave-One-Outの適用例

上記のk-FoldやLeave-One-Outのクラスは引数やメソッドに共通点があるため<br>
ほぼ同じようにモデル構築に適用することができる。

iris (機械学習でよく使われるあやめのデータセット) を読み込む。

In [10]:
iris = load_iris()
X_iris = pd.DataFrame(iris.data, columns=iris.feature_names)
y_iris = iris.target

サンプルサイズは150である。<br>
そして説明変数としてとしてsepal length (がくの長さ), sepal width (がくの幅), petal length (花びらの長さ), petal width (花びらの幅)がある。

In [11]:
X_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


目的変数はあやめの種類である。0: setosa, 1: versicolor, 2: virginica

In [12]:
y_iris

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

サンプルサイズと説明変数の数は`shape`に保存されている。

In [13]:
X_iris.shape

(150, 4)

続けて、例としてStratifiedShuffleSplitを用いてSVMによるクラス分類を行ってみる。

In [14]:
sss = StratifiedShuffleSplit(n_splits=4, test_size=0.5, random_state=0)
print(sss)
for train_index, validation_index in sss.split(X_iris, y_iris):
        print("TRAIN:", train_index, "VALIDATION:", validation_index)

StratifiedShuffleSplit(n_splits=4, random_state=0, test_size=0.5,
            train_size=None)
TRAIN: [ 16  69  15   4  78 138 111  10  93  45  74  58 106  22  56  28 107  27
  94  72  66  33 143  87  96 115  73  84  26 126  11  91 128 105  79  48
   7 148  31 119  59 124  38  57  95 101  83 137 112  52  92  30  63  42
  14 108 125 122 141  32 140  35  76  41   2  18 146 135 127 116  80  29
 104  82  34] VALIDATION: [139  65 145   6 129  25  85  23 118  64  17 121  71  39  67  36 131 149
  24   0  89   8 136 110 132 147 117   9 130  75 134 144  97 114  19  43
  49  21  50  86  37  20  61  81   5 123  44  99  77 102  98   3 142  40
  88  60  12 103  53 109  90 133  70 100  13  47  54   1  51  68 113  62
 120  46  55]
TRAIN: [  7  10 141   6  94  31 113 140 108  11 128  96 149 110  98   4 101  44
   5   2 144 102 112  86  41  20  59 118 148 115  99 132  88  57 105 103
  83  45 138  62  74  81  52  13 114  67  40  47  82  33 106  38  18 135
  63  75  79  37  55  72  70 111  95 142  15  64

pandasのデータフレームおよびnumpyのarrayはどちらもインデックスを指定することで特定の行や要素だけ取り出すことができる。<br>試しにわざとbreakを入れて一つ目のfoldのTraining set setだけ取り出してみる。

In [15]:
sss = StratifiedShuffleSplit(n_splits=4, test_size=0.5, random_state=0)
print(sss)
for train_index, validation_index in sss.split(X_iris, y_iris):
        print("TRAIN:", train_index)
        print(X_iris.loc[train_index, :])
        print(y_iris[train_index])
        break

StratifiedShuffleSplit(n_splits=4, random_state=0, test_size=0.5,
            train_size=None)
TRAIN: [ 16  69  15   4  78 138 111  10  93  45  74  58 106  22  56  28 107  27
  94  72  66  33 143  87  96 115  73  84  26 126  11  91 128 105  79  48
   7 148  31 119  59 124  38  57  95 101  83 137 112  52  92  30  63  42
  14 108 125 122 141  32 140  35  76  41   2  18 146 135 127 116  80  29
 104  82  34]
     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
16                 5.4               3.9                1.3               0.4
69                 5.6               2.5                3.9               1.1
15                 5.7               4.4                1.5               0.4
4                  5.0               3.6                1.4               0.2
78                 6.0               2.9                4.5               1.5
138                6.0               3.0                4.8               1.8
111                6.4               2.7      

この性質を利用してfoldごとにモデルを構築し予測するスクリプトは以下のようになる。

In [16]:
sss = StratifiedShuffleSplit(n_splits=4, test_size=0.25, random_state=0)
for train_index, validation_index in sss.split(X_iris, y_iris):
        X_train, y_train = X_iris.loc[train_index, :], y_iris[train_index]
        X_test, y_test = X_iris.loc[validation_index, :], y_iris[validation_index]
        svc = svm.SVC()
        svc.fit(X_train, y_train)
        print(svc.predict(X_test))

[0 0 0 0 1 1 1 0 1 2 2 2 1 2 1 0 0 2 0 1 2 1 1 0 2 0 0 1 2 1 0 1 2 2 0 1 2
 2]
[0 0 2 2 1 0 0 2 2 2 1 0 0 1 1 1 2 1 2 1 2 1 1 0 1 0 2 1 2 0 1 0 1 0 0 0 2
 2]
[1 2 0 2 0 0 1 2 0 1 1 2 1 1 0 2 2 2 1 0 2 1 2 1 0 0 0 2 0 2 2 0 1 0 1 2 1
 1]
[0 0 1 0 2 0 2 2 1 2 1 0 0 1 2 1 2 0 0 1 0 1 2 1 2 2 2 0 2 1 0 2 1 1 2 1 2
 0]


さらにSVCクラスのscoreメソッドを用いると実測値(観測値)と予測値を比較し正答率を計算することができる。

In [17]:
skf = StratifiedKFold(n_splits=10)
for train_index, validation_index in skf.split(X_iris, y_iris):
        X_train, y_train = X_iris.loc[train_index, :], y_iris[train_index]
        X_test, y_test = X_iris.loc[validation_index, :], y_iris[validation_index]
        svc = svm.SVC()
        svc.fit(X_train, y_train)
        print(svc.score(X_test, y_test))

1.0
0.933333333333
1.0
1.0
1.0
0.933333333333
0.933333333333
1.0
1.0
1.0


これらの一連の工程は4に参照するcross_val_predictやcross_val_scoreを用いることで簡単に行うことができる。

### 4. Cross validtionおよびモデル作成をし予測まで一括で行う関数

#### 4.1 cross_val_predict

[cross_val_predict](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html#sklearn.model_selection.cross_val_predict)を用いるとkFoldあるいはLeave-one-out cross validationを行いValidation setとして予測した値を求めることができる。<br>デフォルトでは3-fold cross validationを行うが上記のLeaveOneOutなどの関数を引数に与えるとそちらを使ってcross validationをすることができる。

In [18]:
svc = svm.SVC()
y_pred = cross_val_predict(svc, X_iris, y_iris)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

#### 4.2 cross_val_score

[cross_val_score](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score)を用いるとkFoldあるいはLeave-one-out cross validationを行いValidation setとした予測値の正答率を求めることができる。<br>
デフォルトでは3-fold cross validationを行うが上記のLeaveOneOutなどの関数を引数に与えるとそちらを使ってcross validationをすることができる。

In [19]:
svc = svm.SVC()
scores = cross_val_score(svc, X_iris, y_iris)
scores

array([ 0.98039216,  0.96078431,  0.97916667])

引数cvにStratifiedKFoldクラスをあたえることで目的変数の各クラスの割合を考慮したTraining set set, Validation set setを作成したk-fold cross validationを行い、<br>その予測値の正答率を求めてみる。

In [20]:
skf = StratifiedKFold(n_splits=10)
svc = svm.SVC()
scores = cross_val_score(svc, X_iris, y_iris, cv=skf)
scores

array([ 1.        ,  0.93333333,  1.        ,  1.        ,  1.        ,
        0.93333333,  0.93333333,  1.        ,  1.        ,  1.        ])

求められたscoresは3の最後に示したスクリプト( In[17] ) で得られた正答率と同じになっている。<br>
したがってcross_val_scoreを用いることで簡単に同じ工程を行うことができるといえる。

#### 4.3 GridSearchCV

[GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV)はSVMのパラメータの最適な値を求めるグリッドサーチのような網羅的な探索が必要な時に使う関数である。<br>デフォルトでは3-fold cross validationを行うが上記のLeaveOneOutなどの関数を引数に与えるとそちらを使ってcross validationをすることができる。

In [21]:
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 5, 10]}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(X_iris, y_iris)
print(f'Best Estimator:\n{clf.best_estimator_}\n')
pd.DataFrame(clf.cv_results_)

Best Estimator:
SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)



,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_C,param_kernel,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.002061,0.000893,0.980000,0.989998,1,linear,"{'C': 1, 'kernel': 'linear'}",1,1.000000,0.979798,0.960784,1.0,0.979167,0.990196,0.000509,0.000340,0.016179,0.008249
1,0.001365,0.000518,0.973333,0.983363,1,rbf,"{'C': 1, 'kernel': 'rbf'}",4,0.980392,0.969697,0.960784,1.0,0.979167,0.980392,0.000149,0.000259,0.009021,0.012548
2,0.001437,0.000270,0.966667,0.983363,5,linear,"{'C': 5, 'kernel': 'linear'}",6,1.000000,0.969697,0.901961,1.0,1.000000,0.980392,0.000560,0.000024,0.046442,0.012548
3,0.000919,0.000289,0.980000,0.983363,5,rbf,"{'C': 5, 'kernel': 'rbf'}",1,0.980392,0.969697,0.960784,1.0,1.000000,0.980392,0.000080,0.000014,0.015925,0.012548
4,0.000831,0.000346,0.973333,0.979996,10,linear,"{'C': 10, 'kernel': 'linear'}",4,1.000000,0.959596,0.921569,1.0,1.000000,0.980392,0.000041,0.000074,0.037154,0.016497
5,0.000871,0.000309,0.980000,0.979996,10,rbf,"{'C': 10, 'kernel': 'rbf'}",1,0.980392,0.959596,0.960784,1.0,1.000000,0.980392,0.000027,0.000048,0.015925,0.016497


best_estimator_フィールドを確認することで最も良いモデルが得られた時のパラメータの値を確認することができる。<br>
今回はkernel='linear', C=1でありデータフレームを見ると確かにmean_test_scoreが最も良い。

### 5. cross_validationモジュールを使用してcross validationを行う

scikit-learn version2.0で廃止される予定だがcross_validationモジュールを使用してもcross validationを行うことができる。<br>
例えばcross_validationモジュールの[KFold](http://scikit-learn.org/0.16/modules/generated/sklearn.cross_validation.KFold.html#sklearn.cross_validation.KFold)クラスを用いることでKFold cross validationを行うことができる。
<br>shuffle=True, random_state=Noneとすることでランダムにすることができる。

In [22]:
from sklearn import cross_validation

/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


上記の赤枠ででている英文はversion 0.2になるとcross_validationモジュールは廃止されることを説明している。<br>
したがって新たに学ぶなら1〜4章で説明したモジュールの関数やクラスを学んだ方が良い。

In [23]:
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13, 14], [15, 16]])
y = np.array([1, 1, 1, 1, 2, 2, 2, 2])
kf = cross_validation.KFold(8, n_folds=2)
print(kf)  
for train_index, validation_index in kf:
        print("TRAIN:", train_index, "VALIDATION:", validation_index)
kf = cross_validation.KFold(8, n_folds=2, shuffle=True, random_state=None)
print(kf)  
for train_index, validation_index in kf:
        print("TRAIN:", train_index, "VALIDATION:", validation_index)

sklearn.cross_validation.KFold(n=8, n_folds=2, shuffle=False, random_state=None)
TRAIN: [4 5 6 7] VALIDATION: [0 1 2 3]
TRAIN: [0 1 2 3] VALIDATION: [4 5 6 7]
sklearn.cross_validation.KFold(n=8, n_folds=2, shuffle=True, random_state=None)
TRAIN: [0 1 4 5] VALIDATION: [2 3 6 7]
TRAIN: [2 3 6 7] VALIDATION: [0 1 4 5]


他にもLeaveOneOut, ShuffleSplit, StratifiedKFoldなどがmodel_selectionモジュールと同様に存在する。<br>
詳しくは[こちら](http://scikit-learn.org/0.16/modules/classes.html#module-sklearn.cross_validation)を参照してほしい。